In [41]:
#FP-growth algorithm


#The class provides the node, node-link, parentnode and children for the parent nodes.
#nodelink used to link similar items
#parent vaiable used to refer to the parent of the node in the tree
#node contains an empty dictionary for the children in the node
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {} 
#increments the count variable with a given amount    
    def inc(self, numOccur):
        self.count += numOccur
#display the tree      
    def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [42]:
#checking the class if it works

rootNode = treeNode('tree',9,None)
rootNode.children['child'] = treeNode('child',13,None)
rootNode.disp()

   tree   9
     child   13


In [43]:
#Constructing the FP-tree

''' createTree function function will be fed dataset and given minSup.
This will generate an FP-tree, this function will go to every transaction of the dataset,
and in the end, it will count the frequency of the items, and information will be stored in the headerTable.
'''

def createTree(dataSet, minSup): 
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable):  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

In [44]:
#Growing the FP-tree as it triverse the dataset more and more
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [45]:
def updateHeader(nodeToTest, targetNode):   # not using recursion
    while (nodeToTest.nodeLink != None):    
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [46]:
import pandas as pd
import numpy as np
import math


def loadSimpDat():
  #uncomment the following two lines for inputing the file name manually
  #filename = input("Enter name of input file: ")
  #transaction_df = open(filename, "r")

  transaction_df = pd.read_csv('/content/drive/MyDrive/GroceryStore.csv', names =['groceries'])
  transaction_df
  transaction_df.index.rename('TID', inplace=True)
  #transaction_df.rename(columns={'MILK,BREAD,BISCUIT' : 'item_list'}, inplace=True)
  trans_df = transaction_df.groceries.str.split(',')
  trans_df
  return trans_df


In [47]:
#The function CreatTree does not take the data in list format. Instead, it needs the data in dictionary format and frequency of the dictionary as value.
#While elements of a set can be modified at any time, elements of the frozenset() remain the same after creation.
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1 #we are assuming all given transactions are unique hence occure only once 
    return retDict

In [48]:
simpDat = loadSimpDat()
simpDat

TID
0                  [MILK, BREAD, BISCUIT]
1      [BREAD, MILK, BISCUIT, CORNFLAKES]
2                 [BREAD, TEA, BOURNVITA]
3               [JAM, MAGGI, BREAD, MILK]
4                   [MAGGI, TEA, BISCUIT]
5                 [BREAD, TEA, BOURNVITA]
6                [MAGGI, TEA, CORNFLAKES]
7            [MAGGI, BREAD, TEA, BISCUIT]
8                [JAM, MAGGI, BREAD, TEA]
9                           [BREAD, MILK]
10    [COFFEE, COCK, BISCUIT, CORNFLAKES]
11    [COFFEE, COCK, BISCUIT, CORNFLAKES]
12             [COFFEE, SUGER, BOURNVITA]
13                  [BREAD, COFFEE, COCK]
14                [BREAD, SUGER, BISCUIT]
15            [COFFEE, SUGER, CORNFLAKES]
16              [BREAD, SUGER, BOURNVITA]
17                 [BREAD, COFFEE, SUGER]
18                 [BREAD, COFFEE, SUGER]
19        [TEA, MILK, COFFEE, CORNFLAKES]
Name: groceries, dtype: object

In [49]:
initSet = createInitSet(simpDat)
initSet

{frozenset({'BISCUIT', 'BREAD', 'MILK'}): 1,
 frozenset({'BISCUIT', 'BREAD', 'CORNFLAKES', 'MILK'}): 1,
 frozenset({'BOURNVITA', 'BREAD', 'TEA'}): 1,
 frozenset({'BREAD', 'JAM', 'MAGGI', 'MILK'}): 1,
 frozenset({'BISCUIT', 'MAGGI', 'TEA'}): 1,
 frozenset({'CORNFLAKES', 'MAGGI', 'TEA'}): 1,
 frozenset({'BISCUIT', 'BREAD', 'MAGGI', 'TEA'}): 1,
 frozenset({'BREAD', 'JAM', 'MAGGI', 'TEA'}): 1,
 frozenset({'BREAD', 'MILK'}): 1,
 frozenset({'BISCUIT', 'COCK', 'COFFEE', 'CORNFLAKES'}): 1,
 frozenset({'BOURNVITA', 'COFFEE', 'SUGER'}): 1,
 frozenset({'BREAD', 'COCK', 'COFFEE'}): 1,
 frozenset({'BISCUIT', 'BREAD', 'SUGER'}): 1,
 frozenset({'COFFEE', 'CORNFLAKES', 'SUGER'}): 1,
 frozenset({'BOURNVITA', 'BREAD', 'SUGER'}): 1,
 frozenset({'BREAD', 'COFFEE', 'SUGER'}): 1,
 frozenset({'COFFEE', 'CORNFLAKES', 'MILK', 'TEA'}): 1}

In [50]:
#The FP-tree
myFPtree, myHeaderTab = createTree(initSet, 3)
myFPtree.disp()

   Null Set   1
     BREAD   11
       BISCUIT   3
         MILK   1
         CORNFLAKES   1
           MILK   1
         SUGER   1
       TEA   3
         BOURNVITA   1
         BISCUIT   1
           MAGGI   1
         MAGGI   1
       MILK   2
         MAGGI   1
       COFFEE   2
         SUGER   1
       SUGER   1
         BOURNVITA   1
     TEA   2
       BISCUIT   1
         MAGGI   1
       CORNFLAKES   1
         MAGGI   1
     COFFEE   4
       BISCUIT   1
         CORNFLAKES   1
       SUGER   1
         BOURNVITA   1
       CORNFLAKES   1
         SUGER   1
       TEA   1
         MILK   1
           CORNFLAKES   1


In the Output, we can see the FP-tree structure of our data set. The most occurring item in the sets has a count of 11 which is Bread. 

Bread count = 11, Biscuit count = 4 => (Bread, Biscuit) occure together atleast 4 times 



In [51]:
#Extracting frequent items from FP-tree.
#bottom up approach: start from leaf nodes to the roof node which null set in our FP-tree.

def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)


#This function gives a list of links from leaves to the root, which will be returned to our next function findprefixpath() 
#and appended to a list named condpats which will help get the results using nodes class, node count and node-link functions.

In [52]:
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [53]:

#Let’s check for the commonly occurred item with MAGGI:
print("The most occuring with MAGGI are: \n")
l  = findPrefixPath('', myHeaderTab['MAGGI'][1])
l



# Write  to file

with open('Items.txt', 'w') as f:
    for item in l:
        f.write("%s\n" % item)


The most occuring with MAGGI are: 



In [54]:
#Let’s check for the commonly occurred item with MILK:
print("The most occuring with MILK are: \n")
findPrefixPath('', myHeaderTab['MILK'][1])

The most occuring with MILK are: 



{frozenset({'BREAD'}): 2,
 frozenset({'BISCUIT', 'BREAD'}): 1,
 frozenset({'BISCUIT', 'BREAD', 'CORNFLAKES'}): 1,
 frozenset({'COFFEE', 'TEA'}): 1}

In [55]:
#Let’s check for the commonly occurred item with TEA:
print("The most occuring with TEA are: \n")
findPrefixPath('', myHeaderTab['TEA'][1])

The most occuring with TEA are: 



{frozenset({'BREAD'}): 3, frozenset({'COFFEE'}): 1}

Here in the above output, we can see that when we asked to function about the occurrence of TEA, it appeared 3 times in the transaction, and the most occurring item with TEA is BREAD.

In [56]:
#Let’s check for the commonly occurred item with BISCUIT:
print("The most occuring with BISCUIT are: \n")
findPrefixPath('', myHeaderTab['BISCUIT'][1])

The most occuring with BISCUIT are: 



{frozenset({'BREAD'}): 3,
 frozenset({'TEA'}): 1,
 frozenset({'BREAD', 'TEA'}): 1,
 frozenset({'COFFEE'}): 1}

We get BISCUIT and BREAD as most occuring. 

In [57]:
#Let’s check for the commonly occurred item with BREAD:
print("The most occuring with BREAD are: \n")
findPrefixPath('', myHeaderTab['BREAD'][1])

The most occuring with BREAD are: 



{}

## Why dont we get most occuring rules when the input is BREAD? Even though we know that BISCUIT and BREAD occure together. 

Because the tree triverses from leaf node to top node, if we start our journey from the top node which is BREAD, there is no rules to be found. That is why to find the occuring items together, we only have to input the leaf nodes to get the whole picture. 

In [58]:
#Let’s check for the commonly occurred item with BREAD:
print("The most occuring with CORNFLAKES are: \n")
findPrefixPath('', myHeaderTab['CORNFLAKES'][1])

The most occuring with CORNFLAKES are: 



{frozenset({'BISCUIT', 'BREAD'}): 1,
 frozenset({'TEA'}): 1,
 frozenset({'COFFEE'}): 1,
 frozenset({'BISCUIT', 'COFFEE'}): 1,
 frozenset({'COFFEE', 'MILK', 'TEA'}): 1}

In [59]:
#Let’s check for the commonly occurred item with BOURNVITA:
print("The most occuring with BOURNVITA are: \n")
findPrefixPath('', myHeaderTab['BOURNVITA'][1])

The most occuring with BOURNVITA are: 



{frozenset({'BREAD', 'TEA'}): 1,
 frozenset({'COFFEE', 'SUGER'}): 1,
 frozenset({'BREAD', 'SUGER'}): 1}

In [60]:
#Let’s check for the commonly occurred item with SUGAR:
print("The most occuring with SUGER are: \n")
findPrefixPath('', myHeaderTab['SUGER'][1])

The most occuring with SUGER are: 



{frozenset({'COFFEE'}): 1,
 frozenset({'BISCUIT', 'BREAD'}): 1,
 frozenset({'COFFEE', 'CORNFLAKES'}): 1,
 frozenset({'BREAD'}): 1,
 frozenset({'BREAD', 'COFFEE'}): 1}

Here we can see from the output and cross verify with the tree output that, BREAD appeared 3 times in transaction with BISCUIT. 

WHy is FP-growth faster? 

This structure helps to find the required frequent set rapidly. Internally FP-growth is an algorithm that does not require candidate generation. It uses an FP-tree data structure that does not require the generation of candidate sets explicitly, making the algorithm work better with large databases.